In [2]:
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import model_from_json
import cv2 as cv

In [3]:
#Создание генератора
datagen = ImageDataGenerator(rescale=1. / 255)

#Загрузка изображений
train_generator = datagen.flow_from_directory(
    'Data_for_lern/train',
    target_size=(300, 300),
    batch_size=5,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    'Data_for_lern/val',
    target_size=(300, 300),
    batch_size=5,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    'Data_for_lern/test',
    target_size=(300, 300),
    batch_size=5,
    class_mode='categorical')

Found 595 images belonging to 4 classes.
Found 163 images belonging to 4 classes.
Found 111 images belonging to 4 classes.


In [4]:
model = Sequential()
model.add(Conv2D(8, (7, 7), input_shape=(300, 300, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
# Размер мини-выборки
batch_size = 5
# Количество изображений для обучения
nb_train_samples = 595
# Количество изображений для проверки
nb_validation_samples = 163
# Количество изображений для тестирования
nb_test_samples = 111

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("model_gen_j_tom.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_gen_h_tom.h5")

Epoch 1/10
119/119 [==============================] - 644s 5s/step - loss: 1.1861 - acc: 0.4672 - val_loss: 0.8765 - val_acc: 0.6563
Epoch 2/10
119/119 [==============================] - 531s 4s/step - loss: 0.8586 - acc: 0.6723 - val_loss: 0.8992 - val_acc: 0.6375
Epoch 3/10
119/119 [==============================] - 458s 4s/step - loss: 0.6566 - acc: 0.7563 - val_loss: 0.5674 - val_acc: 0.7875
Epoch 4/10
119/119 [==============================] - 458s 4s/step - loss: 0.5336 - acc: 0.7815 - val_loss: 0.3830 - val_acc: 0.8688
Epoch 5/10
119/119 [==============================] - 504s 4s/step - loss: 0.5280 - acc: 0.8017 - val_loss: 0.4040 - val_acc: 0.8375
Epoch 6/10
119/119 [==============================] - 458s 4s/step - loss: 0.3815 - acc: 0.8555 - val_loss: 0.3969 - val_acc: 0.8438
Epoch 7/10
119/119 [==============================] - 460s 4s/step - loss: 0.3522 - acc: 0.8790 - val_loss: 0.3310 - val_acc: 0.8563
Epoch 8/10
119/119 [==============================] - 605s 5s/step - 

In [7]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 72.73%


In [3]:
#download net
#загружаем нейронную сеть
json_file = open("save_model/model_gen_j_tom83.json", "r")
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)
model.load_weights("save_model/model_gen_h_tom_83.h5")

#компилиркем нейроннуюсеть
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
#url_img = '17.tif'
#url_img = 'soya.jpg'
#img = cv.imread(url_img)
#img = cv.resize(img, (240, 245))
#img = img / 255
#img = np.expand_dims(img, axis=0)
cap = cv.VideoCapture('C:/Users/Sergey/Pictures/Dataset_fablab/Test_Videos/1.mp4')

#result = model.predict(img)
#print(int(result[0][0]))

while True:
    _, frame = cap.read()
    
    img = cv.resize(frame, (300, 300))
    frame = cv.resize(frame, (500, 500))
    img = img / 255
    img = np.expand_dims(img, axis=0)
    
    result = model.predict(img)
    print('----------------')
    print('Вероятности:')
    print('flower', result[0][0])
    print('other grass', result[0][1])
    print('tomatoes', result[0][2])
    print('strawberry', result[0][3])
    print('++++++++++++++++')
    
    cv.putText(frame, 'flower - ' + str(result[0][0]), (20, 20), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv.putText(frame, 'other grass - ' + str(result[0][1]), (20, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv.putText(frame, 'tomatoes - ' + str(result[0][2]), (20, 80), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv.putText(frame, 'strawberry - ' + str(result[0][3]), (20, 110), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    cv.imshow('img', frame)
    if cv.waitKey(30) == ord('q'):
          break
cap.release()
cv.destroyAllWindows()

----------------
Вероятности:
flower 0.0053798347
other grass 0.08537029
tomatoes 0.6478088
strawberry 0.26144114
++++++++++++++++
----------------
Вероятности:
flower 0.005353094
other grass 0.0850123
tomatoes 0.6627327
strawberry 0.24690187
++++++++++++++++
----------------
Вероятности:
flower 0.0048036776
other grass 0.06563556
tomatoes 0.69656277
strawberry 0.232998
++++++++++++++++
----------------
Вероятности:
flower 0.0050966865
other grass 0.10514562
tomatoes 0.61598265
strawberry 0.273775
++++++++++++++++
----------------
Вероятности:
flower 0.0052572554
other grass 0.15827961
tomatoes 0.5128341
strawberry 0.32362914
++++++++++++++++
----------------
Вероятности:
flower 0.007078228
other grass 0.20450751
tomatoes 0.44286948
strawberry 0.34554476
++++++++++++++++
----------------
Вероятности:
flower 0.0076178396
other grass 0.19259633
tomatoes 0.47612205
strawberry 0.3236638
++++++++++++++++
----------------
Вероятности:
flower 0.0071252864
other grass 0.22918265
tomatoes 0.426

----------------
Вероятности:
flower 0.00016352952
other grass 0.0005020365
tomatoes 0.8065346
strawberry 0.19279994
++++++++++++++++
----------------
Вероятности:
flower 0.000110031964
other grass 0.00038465406
tomatoes 0.7842047
strawberry 0.21530055
++++++++++++++++
----------------
Вероятности:
flower 7.952961e-05
other grass 0.00024709138
tomatoes 0.81936234
strawberry 0.180311
++++++++++++++++
----------------
Вероятности:
flower 9.808804e-05
other grass 0.00043607363
tomatoes 0.7467257
strawberry 0.25274014
++++++++++++++++
----------------
Вероятности:
flower 7.7373035e-05
other grass 0.00041469088
tomatoes 0.6488571
strawberry 0.35065076
++++++++++++++++
----------------
Вероятности:
flower 0.00011935522
other grass 0.000597899
tomatoes 0.6238813
strawberry 0.37540147
++++++++++++++++
----------------
Вероятности:
flower 8.98341e-05
other grass 0.0005638564
tomatoes 0.62359226
strawberry 0.37575412
++++++++++++++++
----------------
Вероятности:
flower 0.00016323836
other grass

----------------
Вероятности:
flower 5.8213203e-09
other grass 2.4561295e-10
tomatoes 0.99837226
strawberry 0.0016277396
++++++++++++++++
----------------
Вероятности:
flower 1.0033484e-08
other grass 5.0505994e-10
tomatoes 0.9977951
strawberry 0.0022049346
++++++++++++++++
----------------
Вероятности:
flower 5.690725e-09
other grass 4.4706622e-10
tomatoes 0.9974305
strawberry 0.0025695125
++++++++++++++++
----------------
Вероятности:
flower 2.6557012e-09
other grass 1.7056688e-10
tomatoes 0.99829644
strawberry 0.0017035705
++++++++++++++++
----------------
Вероятности:
flower 2.1501414e-09
other grass 1.7569966e-10
tomatoes 0.9979773
strawberry 0.00202268
++++++++++++++++
----------------
Вероятности:
flower 1.807113e-08
other grass 2.303801e-09
tomatoes 0.99241924
strawberry 0.0075807995
++++++++++++++++
----------------
Вероятности:
flower 3.5598426e-08
other grass 4.266052e-09
tomatoes 0.9919003
strawberry 0.008099728
++++++++++++++++
----------------
Вероятности:
flower 1.961304

----------------
Вероятности:
flower 0.00038211964
other grass 0.0021031348
tomatoes 0.05633378
strawberry 0.9411809
++++++++++++++++
----------------
Вероятности:
flower 0.00037994471
other grass 0.0022625623
tomatoes 0.06026038
strawberry 0.9370971
++++++++++++++++
----------------
Вероятности:
flower 0.0005345295
other grass 0.0032417218
tomatoes 0.06976328
strawberry 0.9264605
++++++++++++++++
----------------
Вероятности:
flower 0.0009571879
other grass 0.0046762843
tomatoes 0.10322325
strawberry 0.89114326
++++++++++++++++
----------------
Вероятности:
flower 0.0009384803
other grass 0.0046057715
tomatoes 0.110952534
strawberry 0.88350314
++++++++++++++++
----------------
Вероятности:
flower 0.0010633764
other grass 0.005088765
tomatoes 0.12614393
strawberry 0.867704
++++++++++++++++
----------------
Вероятности:
flower 0.0010309785
other grass 0.0038668036
tomatoes 0.16593881
strawberry 0.82916343
++++++++++++++++
----------------
Вероятности:
flower 0.0009199983
other grass 0.0

In [21]:
url_img = '1.jpg'
frame = cv.imread(url_img)
img = cv.resize(frame, (300, 300))
frame= cv.resize(frame, (500, 500))
img = img / 255
img = np.expand_dims(img, axis=0)
#cap = cv.VideoCapture('C:/Users/Sergey/Pictures/Dataset_fablab/Test_Videos/1.mp4')

result = model.predict(img)
print(result)

#max = 0
#for i in result[0]:
##    if (i > max):
#        max = i

#for i in range(len(result[0])):
#    if result[0][i] == max:
#        print(i + 1)
cv.putText(frame, 'flower - ' + str(result[0][0]), (20, 20), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
cv.putText(frame, 'other grass - ' + str(result[0][1]), (20, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
cv.putText(frame, 'tomatoes - ' + str(result[0][2]), (20, 80), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
cv.putText(frame, 'strawberry - ' + str(result[0][3]), (20, 110), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    

while True:
    
    cv.imshow('img', frame)
    if cv.waitKey(30) == ord('q'):
          break
cv.destroyAllWindows()

[[0.00214103 0.01791735 0.04234162 0.9376    ]]
